# CNM_CycNucMed — Clean Colab/Jupyter Runner (run_pipeline)

**CNM_CycNucMed** is a lightweight runner for executing the IAEA cyclotron and nuclear medicine data pipeline from Google Colab or Jupyter environments.
It provides a clean, reproducible workflow for setting up the environment, validating the installation, and generating structured PDF reports via the run_pipeline entrypoint without relying on CLI tools or interactive argument parsing.

The notebook is designed for research, analysis, and reproducible reporting, allowing users to move from raw data to publication-ready outputs with minimal setup friction.

**Cloning or Updating the Repository**

This step ensures that the project source code is available locally and up to date.

**What it does**

*   Clones the GitHub repository if it is not yet present
*   Pulls the latest changes if the repository already exists
*   Moves the working directory into the project folder

**Why this is needed**

You must have the current source code locally before installing dependencies or running the pipeline.
This step guarantees that you are always working on the latest version of the project.

**Notes**
*   If the repository is private, cloning requires a GitHub Personal Access Token (PAT).
*   The script includes commented instructions for securely entering a token if authentication fails.

In [1]:
from google.colab import userdata

# =========================
# Clone / pull + cd
# =========================
import os, subprocess, pathlib, sys
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LOCATIONIQ_API_KEY"] = userdata.get("LOCATIONIQ_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")

%env IAEA_GEOCODER_PROVIDER=locationiq

# =========================
# CNM_CycNucMed — Colab bootstrap (clone/pull + install so `import iaea_project` works)
# =========================

REPO_URL = "https://github.com/tommasocarzaniga/CNM_CycNucMed.git"
REPO_DIR = "/content/CNM_CycNucMed"  # absolute path = more reliable in Colab

def sh(cmd, cwd=None):
    print(">>", cmd)
    subprocess.check_call(cmd, shell=True, cwd=cwd)

# 1) Clone or pull
if not os.path.exists(REPO_DIR):
    sh(f"git clone {REPO_URL} {REPO_DIR}")
else:
    sh("git pull", cwd=REPO_DIR)

# 2) cd into repo
%cd /content/CNM_CycNucMed
print("Now in:", pathlib.Path().resolve())

# 3) Install in editable mode (critical for src/ layout: enables `import iaea_project`)
sh("pip -q install -e .", cwd=REPO_DIR)

# 4) (Optional safety) ensure src is on sys.path in case packaging is misconfigured
src_path = str(pathlib.Path(REPO_DIR) / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# 5) Quick sanity check
import iaea_project
print("iaea_project import OK from:", iaea_project.__file__)


env: IAEA_GEOCODER_PROVIDER=locationiq
>> git pull
/content/CNM_CycNucMed
Now in: /content/CNM_CycNucMed
>> pip -q install -e .
iaea_project import OK from: /content/CNM_CycNucMed/src/iaea_project/__init__.py


**Installing Dependencies and Package (Editable Mode)**

This step installs all required Python dependencies and installs the project itself in editable mode, enabling active development without repeated reinstalls.

**What it does**

*   Upgrades core packaging tools (pip, setuptools, wheel)
*   Installs all dependencies listed in requirements.txt
*   Installs the project with pip install -e ., linking the local source code directly to the environment

**Why this is needed**

Editable installation allows you to:
*   Modify source files and immediately test changes
*   Avoid reinstalling the package after every code update
*   Ensure imports always reflect the current working version of the project

**When to run it**

Run this step once during initial setup and again if you:
*   Change dependencies
*   Modify pyproject.toml or packaging configuration
*   Set up a new environment (machine, venv, container)

In [2]:
import sys, subprocess, pathlib

def pip(cmd: str):
    print(">> pip", cmd)
    subprocess.check_call([sys.executable, "-m", "pip"] + cmd.split())

pip("install -U pip setuptools wheel")

root = pathlib.Path('.').resolve()
if (root / 'requirements.txt').exists():
    pip(f"install -r {root/'requirements.txt'}")

# Editable install (pyproject.toml / src layout supported)
pip("install -e .")


>> pip install -U pip setuptools wheel
>> pip install -r /content/CNM_CycNucMed/requirements.txt
>> pip install -e .


**Verifying the Pipeline Entrypoint**

This step validates that the project is correctly installed and that the main pipeline function (run_pipeline) is available.

It performs three checks:
*   Confirms the imported module path (to ensure the correct environment/package is being used)
*   Reloads the module to pick up recent code changes without restarting the session
*   Asserts that run_pipeline exists, failing early if the installation or package structure is incorrect

**Why this is needed**

This verification prevents subtle development errors such as:
*   Importing the wrong package version
*   Changes in pipeline.py not being reflected in the runtime
*   Missing or misnamed entrypoint functions

**When to use it**

Use this step during development, debugging, or notebook-based workflows to ensure the pipeline is correctly wired before running long jobs.

In [3]:
import importlib
import iaea_project.pipeline as pipeline

# Force reload in case you edited files after the first import
pipeline = importlib.reload(pipeline)

print("pipeline module:", pipeline.__file__)
print("Has run_pipeline:", hasattr(pipeline, "run_pipeline"))
assert hasattr(pipeline, "run_pipeline"), (
    "run_pipeline() not found. Make sure src/iaea_project/pipeline.py defines run_pipeline and that you re-ran the install cell."
)


pipeline module: /content/CNM_CycNucMed/src/iaea_project/pipeline.py
Has run_pipeline: True


**Installing Browser Dependencies (Playwright)**

This command installs Chromium and all required system dependencies for Playwright.
It is required for features that rely on browser automation, such as:
*   Scraping dynamic websites (JavaScript-rendered pages)
*   Automated page rendering for data extraction
*   Generating screenshots or PDFs from HTML content

**Why this step is needed**

Playwright controls a real browser. Without installing Chromium and its dependencies:
*   The pipeline may fail when accessing dynamic web content
*   Rendering-based extraction and automation will not work
*   Errors like "browser executable not found" can occur

**When to run it**

Run this once during environment setup (e.g. after cloning the repository or setting up a new machine/container).

In [4]:
# =========================
# Dependencies (Colab)
# =========================

# Install Chromium + system deps in one go (recommended)
!playwright install --with-deps chromium



Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-freefont-ttf is already the newest version (20120503-10build1

**Running the Pipeline via Python API**

This module allows you to execute the IAEA cyclotron reporting pipeline programmatically from Python.
The run_pipeline function runs the full data processing workflow and returns the file path of the generated PDF report.

What it does:
*   Executes the data pipeline (data loading, processing, summarization, and report generation)
*   Supports running for all countries or a selected subset
*   Returns the absolute path to the generated PDF file

In [5]:
from iaea_project.pipeline import run_pipeline

# Run all countries:
# pdf_path = run_pipeline()

# Run selected countries:
pdf_path = run_pipeline(["China"], enable_llm=True, skip_scrape=False, enable_web_enrichment=True)  # edit as needed

print("Generated PDF at:", pdf_path)


[DEBUG] China ctx found: True facility: Institute of Applied Chemistry
[DEBUG] China top_site_blurb: 'The Institute of Applied Chemistry, located in Beijing, China, operates two cyclotrons manufactured by The Cyclotron Corporation (TCC). These cyclotrons are models CS-22 and CS-30, providing proton e
[DEBUG] sections: 1
[DEBUG] section keys: dict_keys(['country', 'summary_md', 'top_site_blurb', 'top_site_ctx', 'tables', 'images'])
[DEBUG] first top_site_blurb: 'The Institute of Applied Chemistry, located in Beijing, China, operates two cyclotrons manufactured by The Cyclotron Corporation (TCC). These cyclotrons are models CS-22 and CS-30, providing proton e
Saved raw:   /content/CNM_CycNucMed/data/raw/iaea_cyclotrons_raw.csv
Saved clean: /content/CNM_CycNucMed/data/processed/iaea_cyclotrons_clean.csv
Saved PDF:   /content/CNM_CycNucMed/outputs/reports/IAEA_Cyclotron_Report.pdf
Generated PDF at: /content/CNM_CycNucMed/outputs/reports/IAEA_Cyclotron_Report.pdf


**cyclotron_run — CLI Wrapper**

*cyclotron_run* is a lightweight command-line interface (CLI) wrapper designed to simplify the execution of cyclotron-related data pipelines and analysis workflows.
It provides a clean, consistent entry point to run complex scripts with clear commands and arguments, making the tool easier to use, automate, and reproduce.

Instead of manually editing scripts or remembering long commands, users can interact with the system through intuitive commands such as:

*   cyclotron_run country --name "Switzerland"
*   cyclotron_run full

The goal of cyclotron_run is to improve:
*   Reproducibility of analyses
*   Usability for non-developers
*   Maintainability of the codebase
*   Scalability as new features and modules are added

In [6]:
#!python scripts/cyclotron_run.py --countries Switzerland Germany
